[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/cataluna84/VisionInterpretability/blob/main/notebooks/cataluna84__segment_2_activation_max.ipynb)

# Segment 2: Activation Maximization - Visualizing Neural Network Features

## Introduction

This notebook reproduces the **activation maximization** visualizations from the groundbreaking
[Distill.pub Circuits](https://distill.pub/2020/circuits/) research thread, specifically:

- [Zoom In: An Introduction to Circuits](https://distill.pub/2020/circuits/zoom-in/)
- [An Overview of Early Vision in InceptionV1](https://distill.pub/2020/circuits/early-vision/)

These papers demonstrate that neural networks learn **interpretable features** at the neuron level,
and we can visualize what each neuron "looks for" using optimization-based techniques.

---

## Theoretical Background

### What is Activation Maximization?

**Activation maximization** is a feature visualization technique that generates an input image
which maximally activates a specific neuron (or channel) in a neural network.

#### The Core Idea

Given a trained neural network $f$ with frozen weights, we want to find an input image $\mathbf{x}^*$
that maximizes the activation of a target neuron $a_{l,k}$ at layer $l$, channel $k$:

$$\mathbf{x}^* = \arg\max_{\mathbf{x}} \, a_{l,k}(f(\mathbf{x}))$$

Where $a_{l,k}(f(\mathbf{x}))$ represents the mean activation of channel $k$ at layer $l$ when
the network processes input $\mathbf{x}$.

#### Optimization Formulation

In practice, we solve this as a gradient-based optimization problem. Starting from a random
or noise image $\mathbf{x}_0$, we iteratively update the image:

$$\mathbf{x}_{t+1} = \mathbf{x}_t + \eta \cdot \nabla_{\mathbf{x}} \, a_{l,k}(f(\mathbf{x}_t))$$

Where:
- $\eta$ is the learning rate
- $\nabla_{\mathbf{x}}$ is the gradient with respect to the input image
- We're performing **gradient ascent** (maximizing, not minimizing)

#### Regularization for Interpretable Images

Without regularization, the optimized images often contain high-frequency noise patterns
that exploit the network but aren't human-interpretable. Common regularization techniques include:

1. **Total Variation (TV) Loss**: Penalizes high-frequency variations
   $$\mathcal{L}_{TV}(\mathbf{x}) = \sum_{i,j} |x_{i+1,j} - x_{i,j}| + |x_{i,j+1} - x_{i,j}|$$

2. **L2 Regularization**: Prevents pixel values from exploding
   $$\mathcal{L}_{L2}(\mathbf{x}) = \|\mathbf{x}\|_2^2$$

3. **Transformation Robustness**: Apply random jitter, rotation, scaling during optimization

4. **Decorrelated Color Space**: Optimize in a decorrelated color space (often Fourier-based)

The final objective with regularization becomes:

$$\mathbf{x}^* = \arg\max_{\mathbf{x}} \left[ a_{l,k}(f(\mathbf{x})) - \lambda_{TV} \mathcal{L}_{TV}(\mathbf{x}) - \lambda_{L2} \mathcal{L}_{L2}(\mathbf{x}) \right]$$

---

### InceptionV1 Architecture Context

InceptionV1 (GoogLeNet) uses **Inception modules** that apply multiple filter sizes in parallel.
The layer naming convention follows:

| Layer Name | Description | Typical Feature Complexity |
|-----------|-------------|---------------------------|
| `mixed3a/b` | Early layers | Edges, colors, textures |
| `mixed4a/b/c/d/e` | Middle layers | Parts, shapes, patterns |
| `mixed5a/b` | Later layers | Objects, high-level concepts |

We focus on **`mixed4a`** — a transition layer where features become more semantically meaningful
while still being relatively interpretable.

---

## 1. Environment Setup

We use **Lucent** — the PyTorch port of Google/OpenAI's **Lucid** library,
created by the same team that wrote the Distill.pub Circuits articles.

In [ ]:
"""Setup: Import libraries and configure environment.

This cell imports all necessary dependencies for activation maximization
visualization using the Lucent library (PyTorch port of Lucid).
"""

# Standard library imports
import warnings
from typing import Optional

# Third-party imports
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from PIL import Image

# Lucent imports for feature visualization
from lucent.optvis import render, param, transform, objectives
from lucent.modelzoo import inceptionv1

# Suppress non-critical warnings for cleaner output
warnings.filterwarnings('ignore', category=UserWarning)

# Configure matplotlib for high-quality figures
plt.rcParams.update({
    'figure.figsize': (12, 8),
    'figure.dpi': 100,
    'axes.titlesize': 12,
    'axes.labelsize': 10,
    'font.family': 'sans-serif',
})

# Device configuration
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")
if DEVICE.type == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 2. Loading InceptionV1 Model

Lucent provides a pre-trained InceptionV1 model with **correctly named layers**
matching the Distill.pub convention (`mixed3a`, `mixed4a`, etc.).

In [ ]:
"""Load the pre-trained InceptionV1 model from Lucent's model zoo.

The model is loaded with ImageNet pre-trained weights and set to
evaluation mode (frozen batch normalization, dropout disabled).
"""


def load_inceptionv1_model(device: torch.device) -> torch.nn.Module:
    """Load and prepare InceptionV1 model for feature visualization.

    Args:
        device: The torch device (cuda/cpu) to load the model onto.

    Returns:
        torch.nn.Module: The pre-trained InceptionV1 model in eval mode.

    Notes:
        The model uses layer names like 'mixed3a', 'mixed4a', etc.
        which correspond to the Inception module outputs.
    """
    print("Loading InceptionV1 model...")
    model = inceptionv1(pretrained=True)
    model = model.to(device)
    model = model.eval()

    # Freeze all parameters (we only optimize the input image)
    for param in model.parameters():
        param.requires_grad = False

    print("Model loaded successfully!")
    return model


# Load the model
model = load_inceptionv1_model(DEVICE)

### Exploring the Model Architecture

Let's examine the available layer names in InceptionV1 to understand
where we can hook into for feature visualization.

In [ ]:
"""Explore InceptionV1 architecture and available layer names.

This helps us understand which layers we can target for
activation maximization visualization.
"""


def print_model_layers(
    model: torch.nn.Module,
    max_depth: int = 1
) -> None:
    """Print the top-level named modules of a PyTorch model.

    Args:
        model: The PyTorch model to inspect.
        max_depth: Maximum depth of nested modules to show.

    Returns:
        None: Prints layer information to stdout.
    """
    print("InceptionV1 Layer Structure:")
    print("=" * 50)

    for name, module in model.named_children():
        # Get output shape info if available
        module_type = type(module).__name__
        print(f"  {name:20s} -> {module_type}")


print_model_layers(model)

print("\n" + "=" * 50)
print("Key layers for visualization:")
print("  - mixed3a/b: Early vision (edges, textures)")
print("  - mixed4a/b/c/d/e: Mid-level features (shapes, patterns)")
print("  - mixed5a/b: High-level features (objects, parts)")

### Understanding mixed4a Layer

Let's examine the `mixed4a` layer specifically to understand
how many neurons (channels) it contains.

In [ ]:
"""Analyze the mixed4a layer structure and channel count.

mixed4a is an Inception module that concatenates outputs from
multiple parallel convolution branches.
"""


def get_layer_output_channels(
    model: torch.nn.Module,
    layer_name: str,
    input_size: tuple = (1, 3, 224, 224)
) -> int:
    """Get the number of output channels for a specific layer.

    Args:
        model: The PyTorch model.
        layer_name: Name of the layer to analyze.
        input_size: Input tensor size for forward pass.

    Returns:
        int: Number of output channels at the specified layer.

    Notes:
        Uses a forward hook to capture the layer's output shape.
    """
    output_shape = None

    def hook_fn(module, input, output):
        nonlocal output_shape
        output_shape = output.shape

    # Find and register hook on the target layer
    for name, module in model.named_modules():
        if name == layer_name:
            handle = module.register_forward_hook(hook_fn)
            break
    else:
        raise ValueError(f"Layer '{layer_name}' not found in model")

    # Run forward pass with dummy input
    with torch.no_grad():
        dummy_input = torch.randn(input_size).to(DEVICE)
        model(dummy_input)

    handle.remove()
    return output_shape[1]  # Channel dimension


# Analyze mixed4a
TARGET_LAYER = 'mixed4a'
num_channels = get_layer_output_channels(model, TARGET_LAYER)

print(f"Layer: {TARGET_LAYER}")
print(f"Number of channels (neurons): {num_channels}")
print(f"We will visualize the first 10 neurons (indices 0-9)")

---

## 3. Single Neuron Visualization

Before generating all 10 visualizations, let's understand the process
by visualizing a single neuron in detail.

### The render_vis Function

Lucent's `render_vis` function handles all the optimization details:

```python
render.render_vis(
    model,           # The neural network
    objective,       # What to maximize (e.g., "mixed4a:0")
    param_f=None,    # Image parameterization (default: FFT-based)
    optimizer=None,  # Optimizer (default: Adam)
    transforms=None, # Data augmentation for robustness
    thresholds=(512,), # Optimization steps
    verbose=False,   # Print progress
    show_image=True, # Display result
)
```

The objective string format is `"layer_name:channel_index"`.

In [ ]:
"""Demonstrate activation maximization for a single neuron.

This cell visualizes neuron 0 of mixed4a layer using Lucent's
render_vis function with default settings.
"""


def visualize_single_neuron(
    model: torch.nn.Module,
    layer_name: str,
    neuron_idx: int,
    image_size: int = 128,
    num_steps: int = 512,
    show: bool = True
) -> np.ndarray:
    """Generate activation maximization visualization for a single neuron.

    Uses gradient ascent to find an input image that maximally activates
    the specified neuron in the network.

    Args:
        model: Pre-trained neural network in eval mode.
        layer_name: Name of the layer containing the target neuron.
        neuron_idx: Index of the neuron (channel) to visualize.
        image_size: Output image resolution (square).
        num_steps: Number of optimization iterations.
        show: Whether to display the result.

    Returns:
        np.ndarray: The generated visualization image (H, W, 3).

    Notes:
        - Uses FFT-based image parameterization for smooth results
        - Applies standard transformation robustness (jitter, scale, rotate)
        - More steps generally produce clearer features
    """
    # Define the optimization objective
    objective = f"{layer_name}:{neuron_idx}"

    print(f"Generating visualization for {objective}...")
    print(f"  Image size: {image_size}x{image_size}")
    print(f"  Optimization steps: {num_steps}")

    # Define image parameterization using FFT for smooth results
    param_f = lambda: param.image(image_size, fft=True, decorrelate=True)

    # Run activation maximization
    images = render.render_vis(
        model,
        objective,
        param_f=param_f,
        thresholds=(num_steps,),
        show_image=show,
        verbose=False,
    )

    # images is a list of images at different thresholds
    # We take the final result
    result_image = images[0][0]  # Shape: (H, W, 3)

    return result_image


# Visualize neuron 0 of mixed4a
print("="*60)
print("SINGLE NEURON VISUALIZATION DEMO")
print("="*60)
demo_image = visualize_single_neuron(
    model,
    layer_name='mixed4a',
    neuron_idx=0,
    image_size=128,
    num_steps=512,
    show=True
)

# Plot the generated visualization with matplotlib
plt.figure(figsize=(8, 8))
plt.imshow(np.clip(demo_image, 0, 1))
plt.title(
    'Activation Maximization: mixed4a:0\n'
    'Image that maximally activates neuron 0',
    fontsize=12,
    fontweight='bold'
)
plt.axis('off')
plt.tight_layout()
plt.show()

### Understanding the Visualization

The generated image shows patterns that **maximally activate** this neuron.
Key observations:

1. **Repeating patterns**: The same motif appears throughout the image
   because this maximizes the spatial average of the neuron's activation

2. **Specific colors/orientations**: The neuron may prefer certain
   color combinations or edge orientations

3. **Abstract but structured**: Unlike random noise, these patterns
   have clear structure that reveals what the neuron "looks for"

---

## 4. Visualizing First 10 Neurons of mixed4a

Now we'll generate activation maximization images for neurons 0-9 of the
`mixed4a` layer and present them in a grid visualization, similar to the
Distill.pub formatting.

In [ ]:
"""Generate activation maximization visualizations for multiple neurons.

This cell creates visualizations for the first 10 neurons of mixed4a
and stores them for later display.
"""


def generate_neuron_visualizations(
    model: torch.nn.Module,
    layer_name: str,
    neuron_indices: list,
    image_size: int = 128,
    num_steps: int = 512
) -> dict:
    """Generate activation maximization images for multiple neurons.

    Args:
        model: Pre-trained neural network.
        layer_name: Target layer name.
        neuron_indices: List of neuron indices to visualize.
        image_size: Output image resolution.
        num_steps: Optimization iterations per neuron.

    Returns:
        dict: Mapping from neuron index to visualization image.

    Example:
        >>> images = generate_neuron_visualizations(
        ...     model, 'mixed4a', range(10)
        ... )
        >>> images[0].shape  # (128, 128, 3)
    """
    visualizations = {}

    # Define image parameterization once (reused for each neuron)
    param_f = lambda: param.image(image_size, fft=True, decorrelate=True)

    total = len(neuron_indices)
    for i, neuron_idx in enumerate(neuron_indices):
        objective = f"{layer_name}:{neuron_idx}"
        print(f"[{i+1}/{total}] Generating {objective}...", end=" ")

        # Generate visualization (suppress display)
        images = render.render_vis(
            model,
            objective,
            param_f=param_f,
            thresholds=(num_steps,),
            show_image=False,
            verbose=False,
        )

        visualizations[neuron_idx] = images[0][0]
        print("Done!")

    return visualizations


# Generate visualizations for first 10 neurons
print("="*60)
print("GENERATING VISUALIZATIONS FOR FIRST 10 NEURONS OF mixed4a")
print("="*60)
print(f"Layer: mixed4a")
print(f"Neurons: 0-9 (10 total)")
print(f"Image size: 128x128")
print(f"Optimization steps: 512")
print("-"*60)

neuron_visualizations = generate_neuron_visualizations(
    model=model,
    layer_name='mixed4a',
    neuron_indices=list(range(10)),
    image_size=128,
    num_steps=512
)

print("-"*60)
print(f"Generated {len(neuron_visualizations)} visualizations!")

### Grid Visualization

Display all 10 neuron visualizations in a clean 2×5 grid layout,
following the Distill.pub visual style.

In [ ]:
"""Display neuron visualizations in a publication-quality grid layout.

Creates a 2x5 grid of activation maximization images with labels,
similar to the Distill.pub Circuits visualization style.
"""


def display_neuron_grid(
    visualizations: dict,
    layer_name: str,
    ncols: int = 5,
    figsize: tuple = (15, 7),
    title: Optional[str] = None
) -> plt.Figure:
    """Display neuron visualizations in a grid layout.

    Args:
        visualizations: Dict mapping neuron indices to images.
        layer_name: Name of the layer for labeling.
        ncols: Number of columns in the grid.
        figsize: Figure size (width, height).
        title: Optional title for the entire figure.

    Returns:
        plt.Figure: The matplotlib figure object.
    """
    # Calculate grid dimensions
    n_images = len(visualizations)
    nrows = (n_images + ncols - 1) // ncols

    # Create figure with constrained layout
    fig, axes = plt.subplots(
        nrows, ncols,
        figsize=figsize,
        constrained_layout=True
    )

    # Flatten axes for easy iteration
    if nrows == 1:
        axes = axes.reshape(1, -1)
    axes_flat = axes.flatten()

    # Plot each visualization
    sorted_indices = sorted(visualizations.keys())
    for ax_idx, neuron_idx in enumerate(sorted_indices):
        ax = axes_flat[ax_idx]
        image = visualizations[neuron_idx]

        # Ensure image is in valid range [0, 1] for display
        image_clipped = np.clip(image, 0, 1)

        ax.imshow(image_clipped)
        ax.set_title(f"{layer_name}:{neuron_idx}", fontsize=11, fontweight='bold')
        ax.axis('off')

    # Hide unused subplots
    for ax_idx in range(len(sorted_indices), len(axes_flat)):
        axes_flat[ax_idx].axis('off')
        axes_flat[ax_idx].set_visible(False)

    # Add main title if provided
    if title:
        fig.suptitle(title, fontsize=14, fontweight='bold', y=1.02)

    return fig


# Display the grid
fig = display_neuron_grid(
    neuron_visualizations,
    layer_name='mixed4a',
    ncols=5,
    figsize=(16, 7),
    title='Activation Maximization: First 10 Neurons of InceptionV1 mixed4a'
)

plt.show()

---

## 5. Analysis and Interpretation

### What Do These Neurons Detect?

Looking at the generated visualizations, we can interpret what each
neuron has learned to detect. Common feature types in `mixed4a` include:

| Feature Type | Characteristics | Example Appearance |
|-------------|-----------------|--------------------|
| **Edges** | Oriented lines, gradients | Parallel stripes |
| **Textures** | Repeating patterns | Grid-like or wave patterns |
| **Colors** | Color combinations | Specific hue gradients |
| **Curves** | Circular arcs, spirals | Curved line segments |
| **Corners** | Angle detection | L-shaped or T-junction patterns |

As noted in the Distill.pub [Early Vision](https://distill.pub/2020/circuits/early-vision/) article:

> *"Early layers detect simple features like edges and colors.
> As we go deeper, features become more complex and semantic."*

In [ ]:
"""Provide detailed view of individual neurons with interpretation.

This cell displays each neuron with more space for detailed examination.
"""


def display_detailed_neurons(
    visualizations: dict,
    layer_name: str,
    neurons_per_row: int = 2
) -> plt.Figure:
    """Display neurons with larger images for detailed analysis.

    Args:
        visualizations: Dict mapping neuron indices to images.
        layer_name: Name of the layer.
        neurons_per_row: Number of neurons to show per row.

    Returns:
        plt.Figure: The matplotlib figure object.
    """
    n_neurons = len(visualizations)
    n_rows = (n_neurons + neurons_per_row - 1) // neurons_per_row

    fig, axes = plt.subplots(
        n_rows, neurons_per_row,
        figsize=(12, 5 * n_rows),
        constrained_layout=True
    )

    axes_flat = axes.flatten() if n_rows > 1 else [axes] if neurons_per_row == 1 else axes

    sorted_indices = sorted(visualizations.keys())
    for ax_idx, neuron_idx in enumerate(sorted_indices):
        if ax_idx >= len(axes_flat):
            break

        ax = axes_flat[ax_idx]
        image = np.clip(visualizations[neuron_idx], 0, 1)

        ax.imshow(image, interpolation='lanczos')
        ax.set_title(
            f"{layer_name}:{neuron_idx}",
            fontsize=14,
            fontweight='bold',
            pad=10
        )
        ax.axis('off')

    # Hide unused axes
    for ax_idx in range(len(sorted_indices), len(axes_flat)):
        if ax_idx < len(axes_flat):
            axes_flat[ax_idx].set_visible(False)

    fig.suptitle(
        'Detailed View: mixed4a Neurons',
        fontsize=16,
        fontweight='bold',
        y=1.01
    )

    return fig


# Show first 4 neurons in detail
subset_viz = {k: neuron_visualizations[k] for k in list(neuron_visualizations.keys())[:4]}
fig = display_detailed_neurons(subset_viz, 'mixed4a', neurons_per_row=2)
plt.show()

---

## 6. Advanced: Custom Optimization Parameters

We can customize the optimization process to generate different styles
of visualizations. Key parameters include:

### Image Parameterization

- **FFT (Fourier) parameterization**: Produces smoother, more natural-looking images
- **Pixel parameterization**: Direct optimization in pixel space (often noisier)
- **Decorrelation**: Optimizes in a decorrelated color space

### Transformations

Random transformations during optimization improve **robustness**:
- **Jitter**: Small random translations
- **Scale**: Random zoom in/out
- **Rotate**: Small random rotations

In [ ]:
"""Demonstrate different parameterization options.

Compare FFT-based vs pixel-based parameterization to show
the importance of proper image parameterization.
"""


def compare_parameterizations(
    model: torch.nn.Module,
    layer_name: str,
    neuron_idx: int,
    image_size: int = 128,
    num_steps: int = 256
) -> tuple:
    """Compare FFT vs pixel parameterization for the same neuron.

    Args:
        model: The neural network model.
        layer_name: Target layer name.
        neuron_idx: Neuron index to visualize.
        image_size: Output image resolution.
        num_steps: Optimization iterations.

    Returns:
        tuple: (fft_image, pixel_image) as numpy arrays.
    """
    objective = f"{layer_name}:{neuron_idx}"

    # FFT parameterization (smooth)
    print(f"Generating with FFT parameterization...")
    param_fft = lambda: param.image(image_size, fft=True, decorrelate=True)
    images_fft = render.render_vis(
        model, objective,
        param_f=param_fft,
        thresholds=(num_steps,),
        show_image=False,
        verbose=False
    )

    # Pixel parameterization (often noisier)
    print(f"Generating with pixel parameterization...")
    param_pixel = lambda: param.image(image_size, fft=False, decorrelate=False)
    images_pixel = render.render_vis(
        model, objective,
        param_f=param_pixel,
        thresholds=(num_steps,),
        show_image=False,
        verbose=False
    )

    return images_fft[0][0], images_pixel[0][0]


# Compare for neuron 5
print("Comparing parameterization methods for mixed4a:5")
print("-" * 50)
fft_img, pixel_img = compare_parameterizations(
    model, 'mixed4a', 5, image_size=128, num_steps=256
)

# Display comparison
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].imshow(np.clip(fft_img, 0, 1))
axes[0].set_title('FFT Parameterization\n(Smoother, more natural)', fontsize=12)
axes[0].axis('off')

axes[1].imshow(np.clip(pixel_img, 0, 1))
axes[1].set_title('Pixel Parameterization\n(More high-frequency noise)', fontsize=12)
axes[1].axis('off')

fig.suptitle('Effect of Image Parameterization on Feature Visualization',
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---

## 7. High-Resolution Visualization

For publication-quality images, we can generate higher resolution
visualizations with more optimization steps.

In [ ]:
"""Generate high-resolution visualization for a selected neuron.

Higher resolution and more optimization steps produce cleaner,
more detailed feature visualizations.
"""


def generate_high_res_visualization(
    model: torch.nn.Module,
    layer_name: str,
    neuron_idx: int,
    image_size: int = 256,
    num_steps: int = 1024
) -> np.ndarray:
    """Generate high-resolution activation maximization image.

    Args:
        model: The neural network model.
        layer_name: Target layer name.
        neuron_idx: Neuron index to visualize.
        image_size: Output resolution (larger = more detail).
        num_steps: Optimization iterations (more = cleaner).

    Returns:
        np.ndarray: High-resolution visualization image.

    Notes:
        Higher resolution requires more VRAM. Reduce image_size
        if you encounter out-of-memory errors.
    """
    objective = f"{layer_name}:{neuron_idx}"
    print(f"Generating high-res visualization for {objective}")
    print(f"  Resolution: {image_size}x{image_size}")
    print(f"  Optimization steps: {num_steps}")
    print("  This may take a minute...")

    param_f = lambda: param.image(image_size, fft=True, decorrelate=True)

    images = render.render_vis(
        model,
        objective,
        param_f=param_f,
        thresholds=(num_steps,),
        show_image=False,
        verbose=False
    )

    return images[0][0]


# Generate high-res visualization for neuron 3
high_res_image = generate_high_res_visualization(
    model,
    layer_name='mixed4a',
    neuron_idx=3,
    image_size=256,
    num_steps=1024
)

# Display
plt.figure(figsize=(10, 10))
plt.imshow(np.clip(high_res_image, 0, 1))
plt.title('High-Resolution Visualization: mixed4a:3\n(256×256, 1024 steps)',
          fontsize=14, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.show()

---

## 8. Summary and Key Takeaways

### What We Learned

1. **Activation Maximization** reveals what patterns maximally activate specific neurons

2. **InceptionV1's mixed4a layer** contains diverse features:
   - Edges and gradients
   - Textures and patterns
   - Color preferences
   - Early shape detectors

3. **Lucent/Lucid** makes feature visualization accessible with:
   - Pre-trained models with correct layer naming
   - FFT-based image parameterization for smooth results
   - Transformation robustness for reliable visualizations

### Connection to Circuits Research

The Distill.pub [Circuits](https://distill.pub/2020/circuits/) project shows that:

> Neural networks contain **interpretable features** (neurons) that combine to form
> **circuits** (connected groups of neurons) which implement meaningful algorithms.

This notebook demonstrated the first step: **visualizing individual features**.
Future work could explore:
- How features combine into circuits
- Feature evolution across layers
- Object-specific vs texture-specific neurons

### References

1. Olah, C., et al. (2020). [Zoom In: An Introduction to Circuits](https://distill.pub/2020/circuits/zoom-in/). *Distill*.
2. Olah, C., et al. (2020). [An Overview of Early Vision in InceptionV1](https://distill.pub/2020/circuits/early-vision/). *Distill*.
3. Olah, C., et al. (2017). [Feature Visualization](https://distill.pub/2017/feature-visualization/). *Distill*.
4. Mordvintsev, A., et al. (2018). [Differentiable Image Parameterizations](https://distill.pub/2018/differentiable-parameterizations/). *Distill*.

In [ ]:
"""Final summary: Display all 10 neurons with enhanced formatting.

This provides a clean, publication-ready summary visualization.
"""

# Create final summary figure
fig = plt.figure(figsize=(18, 8))

# Title and description
fig.suptitle(
    'Activation Maximization: First 10 Neurons of InceptionV1 mixed4a Layer',
    fontsize=16,
    fontweight='bold',
    y=0.98
)

# Create grid
gs = GridSpec(2, 5, figure=fig, hspace=0.3, wspace=0.1)

for idx in range(10):
    row = idx // 5
    col = idx % 5

    ax = fig.add_subplot(gs[row, col])
    image = np.clip(neuron_visualizations[idx], 0, 1)

    ax.imshow(image)
    ax.set_title(f'Neuron {idx}', fontsize=11, fontweight='bold')
    ax.axis('off')

# Add methodology note
fig.text(
    0.5, 0.02,
    'Generated using Lucent library with FFT parameterization, 512 optimization steps',
    ha='center',
    fontsize=10,
    style='italic',
    alpha=0.7
)

plt.tight_layout(rect=[0, 0.05, 1, 0.95])
plt.show()

print("\n" + "="*60)
print("NOTEBOOK COMPLETE")
print("="*60)
print("Successfully reproduced activation maximization visualizations")
print("for the first 10 neurons of InceptionV1's mixed4a layer.")
print("\nKey libraries used:")
print("  - torch-lucent (PyTorch port of Lucid)")
print("  - matplotlib (visualization)")
print("  - numpy (array operations)")